In [1]:
import pandas as pd
import cloudscraper
from io import BytesIO
import time

In [2]:
def KRX_Crolling(otp_form_data):
  # generate.cmd 요청 주소 
  otp_url ='http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd' 
  # download.cmd 요청 주소
  csv_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

  # scraper 객체 
  scraper = cloudscraper.create_scraper()

  # form data와 함께 요청 
  # response의 text 에 담겨있는 otp 코드 가져오기
  otp = scraper.post(otp_url, params=otp_form_data).text

  date = otp_form_data['trdDd'][2:]

  csv_form_data = scraper.post(csv_url, params={'code': otp})
  stock_df = pd.read_csv(BytesIO(csv_form_data.content), encoding='euc-kr')

  if otp_form_data['money'] == '3':
    stock_df.to_csv('KRX/kospi/' + date + '.csv',index=False, encoding='euc-kr')
  else :
    stock_df.to_csv('KRX/marcap/' + date + '.csv',index=False, encoding='euc-kr')

  return 

# form data 
kospi_form_data = {
  'locale': 'ko_KR',
  'idxIndMidclssCd': '02',
  'trdDd': '20240312',
  "share": '2',
  'money': '3',
  "csvxls_isNo": 'false',
  "name": 'fileDown',
  "url": 'dbms/MDC/STAT/standard/MDCSTAT00101'
}

marcap_form_data = {
  'locale': 'ko_KR',
  'mktId': 'ALL',
  'trdDd': '20240312',
  'share': '1',
  'money': '1',
  'csvxls_isNo': 'false',
  'name': 'fileDown',
  'url': 'dbms/MDC/STAT/standard/MDCSTAT01501'
}

In [3]:
KRX_Crolling(kospi_form_data)
time.sleep(0.1)
KRX_Crolling(marcap_form_data)